In [1]:
import pandas as pd

In [2]:
from big_picture.classifier import Classifier

In [3]:
model = Classifier()

In [12]:
data = pd.read_json('../raw_data/news_category_data.json', lines=True)
data

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [10]:
from big_picture.pre_processor import pre_process

In [7]:
%load_ext autoreload
%autoreload 2

In [14]:
pre_processed_data = pre_process(data, source='train', params={'lemmatize': False}, printed=True)

Preparing pre_processed_text column
Lowering the cases
Mapping labels
-------------------------
Minor pre-processing done
-------------------------
Counting numbers
Revealing emotions
Analyzing vocabulary bank account!
-------------------------
New features added
-------------------------
Removing digits
Removing pesky dots
Disabling emojis
-------------------------
Excess data removed
-------------------------
Tokenizing
Checking for stopwords
-------------------------
Data pre-processed
-------------------------


In [51]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

y = ohe.fit_transform(pre_processed_data[['label']]).toarray()
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [78]:
from tensorflow.keras import models
from tensorflow.keras import layers

def initialize_class_bert_dropout(shape, output):
    
    model = models.Sequential()
    
    model.add(layers.Dense(700, activation='relu', input_dim=shape))
    #model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(512, activation='relu'))
    #model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(256, activation='relu'))
    
    model.add(layers.Dense(128, activation='relu'))   
    
    model.add(layers.Dense(64, activation='relu'))
    
    model.add(layers.Dense(32, activation='relu'))
    
    model.add(layers.Dense(output, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [4]:
# Embedded stringe
embeddings = pd.read_csv('../big_picture/data/embeddings_200k.csv')

In [69]:
model.model = initialize_class_bert_dropout(768, 16)

In [79]:
model1 = Classifier()
model1.model = initialize_class_bert_dropout(768, 16)

In [70]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=15)

In [47]:
embeddings = embeddings.drop(columns='Unnamed: 0')

In [49]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.556587,-1.730802,0.565383,0.147426,-0.014432,-0.404302,-0.217101,-0.386556,0.452206,1.035194,...,-0.104107,0.873444,0.321066,1.288373,0.182126,-0.084114,-0.205512,-0.034212,0.356344,-0.120589
1,0.420517,0.315536,0.600557,-0.438315,-0.295465,-0.373603,0.222069,-0.523006,1.455598,-0.317190,...,0.074225,-0.834088,0.216602,0.820604,0.360761,0.244211,0.142351,0.626780,-0.310830,-0.820157
2,-0.213008,-0.831775,0.318774,-0.510296,-0.321338,-1.113930,0.169799,-0.350170,-0.028248,0.239748,...,-0.347566,-0.848967,1.017001,0.384794,-0.122068,-0.858027,0.074009,-0.777332,-0.044370,-0.005434
3,-0.331016,0.028823,-0.067903,-1.094421,-1.116765,-0.424198,-0.247861,0.491398,1.089847,-1.060441,...,-0.346633,0.559534,-1.272676,1.694678,0.189955,-0.056552,0.082049,-0.059729,0.273527,-0.356776
4,-0.863933,-0.043071,0.273481,-0.737110,-0.241176,-0.027584,-0.312297,-0.050383,0.203471,0.606433,...,-1.224242,0.212249,-1.223779,0.090633,-0.152032,-0.568693,0.171142,-0.451859,0.057632,0.431622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200848,-1.102570,-0.354107,0.119313,-1.576579,-0.017725,-0.855054,0.947966,0.429000,0.492465,-0.414553,...,-0.173267,-0.731688,-0.353631,0.315879,0.640798,0.529474,0.113282,0.059109,-0.253340,-0.188106
200849,-0.532881,-0.593179,0.556835,-0.762534,-0.687641,-0.610300,-0.360306,0.148103,0.705661,0.425132,...,-0.284688,-0.397910,-0.507344,0.324770,0.043224,-0.739184,-0.799036,-0.414117,0.541927,-0.406548
200850,-0.009099,-0.431355,-0.024914,-0.295087,-0.668326,-0.086673,-0.513776,-0.497617,0.294848,0.326768,...,-0.808323,0.643555,0.074096,0.296579,0.077952,-0.466608,0.104668,-0.880044,0.597700,-0.689383
200851,-0.093480,-1.100077,-0.535262,-0.344463,-0.022475,-0.427429,0.050615,-0.148834,0.114921,-0.042271,...,-0.875505,0.759278,-0.496298,1.150534,-0.149261,-0.616003,-0.017094,-0.261166,0.872171,-1.047837


In [80]:
model1.model.fit(
    embeddings,           
    y,
    epochs=30,
    validation_split=0.25,
    batch_size=16,
    callbacks=[es],
    verbose=1
    )

Epoch 1/30
9415/9415 [==============================] - 51s 5ms/step - loss: 1.4576 - accuracy: 0.5567 - val_loss: 1.3008 - val_accuracy: 0.6132
Epoch 2/30
9415/9415 [==============================] - 51s 5ms/step - loss: 1.2861 - accuracy: 0.6043 - val_loss: 1.2154 - val_accuracy: 0.6405
Epoch 3/30
9415/9415 [==============================] - 50s 5ms/step - loss: 1.2249 - accuracy: 0.6208 - val_loss: 1.1771 - val_accuracy: 0.6469
Epoch 4/30
9415/9415 [==============================] - 53s 6ms/step - loss: 1.1853 - accuracy: 0.6320 - val_loss: 1.1948 - val_accuracy: 0.6436
Epoch 5/30
9415/9415 [==============================] - 52s 5ms/step - loss: 1.1512 - accuracy: 0.6416 - val_loss: 1.1437 - val_accuracy: 0.6505
Epoch 6/30
9415/9415 [==============================] - 51s 5ms/step - loss: 1.1209 - accuracy: 0.6502 - val_loss: 1.1924 - val_accuracy: 0.6472
Epoch 7/30
9415/9415 [==============================] - 52s 6ms/step - loss: 1.0923 - accuracy: 0.6591 - val_loss: 1.1339 - val_ac

In [71]:
model.model.fit(
    embeddings,           
    y,
    epochs=30,
    validation_split=0.25,
    batch_size=32,
    callbacks=[es],
    verbose=1
    )

Epoch 1/30
4708/4708 [==============================] - 24s 5ms/step - loss: 1.4087 - accuracy: 0.5661 - val_loss: 1.2188 - val_accuracy: 0.6290
Epoch 2/30
4708/4708 [==============================] - 20s 4ms/step - loss: 1.2393 - accuracy: 0.6126 - val_loss: 1.1208 - val_accuracy: 0.6545
Epoch 3/30
4708/4708 [==============================] - 22s 5ms/step - loss: 1.1786 - accuracy: 0.6296 - val_loss: 1.1259 - val_accuracy: 0.6588
Epoch 4/30
4708/4708 [==============================] - 22s 5ms/step - loss: 1.1339 - accuracy: 0.6411 - val_loss: 1.1930 - val_accuracy: 0.6370
Epoch 5/30
4708/4708 [==============================] - 22s 5ms/step - loss: 1.0942 - accuracy: 0.6500 - val_loss: 1.1920 - val_accuracy: 0.6404
Epoch 6/30
4708/4708 [==============================] - 25s 5ms/step - loss: 1.0556 - accuracy: 0.6608 - val_loss: 1.1396 - val_accuracy: 0.6510
Epoch 7/30
4708/4708 [==============================] - 28s 6ms/step - loss: 1.0205 - accuracy: 0.6709 - val_loss: 1.1657 - val_ac

In [81]:
model.model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 700)               538300    
_________________________________________________________________
dense_34 (Dense)             (None, 300)               210300    
_________________________________________________________________
dense_35 (Dense)             (None, 150)               45150     
_________________________________________________________________
dense_36 (Dense)             (None, 75)                11325     
_________________________________________________________________
dense_37 (Dense)             (None, 30)                2280      
_________________________________________________________________
dense_38 (Dense)             (None, 16)                496       
Total params: 807,851
Trainable params: 807,851
Non-trainable params: 0
________________________________________________

In [ ]:
from sklearn.metrics import confusion_matrix

cm_array = confusion_matrix(y, results)

In [60]:
y_true = y_true.map(val_dict)

print(classification_report(y_true, model.model.classes))

NameError: name 'classification_report' is not defined

In [59]:
    

# Classification report
if report:
    return 

# Confusion Matrix plot
else:
    cm_array = confusion_matrix(y_true, self.classes)

    df_cm = pd.DataFrame(cm_array, 
                         index = [i for i in labels],
                         columns = [i for i in labels])
    plt.figure(figsize = (15,8))
    return sns.heatmap(df_cm, annot=True)

NameError: name 'y_true' is not defined

In [84]:
divide_data = pd.read_csv('../big_picture/data/newsAPI_30k_data_unprocessed.csv')

In [99]:
divide_data_100 = divide_data.iloc[0:1, :]

In [117]:
model1.divide_labels(divide_data, source='prepared', params={'cat_mapping': False, 'lemmatize': False}, printed=True)

Preparing pre_processed_text column
Lowering the cases
-------------------------
Minor pre-processing done
-------------------------
Counting numbers
Revealing emotions
Analyzing vocabulary bank account!
-------------------------
New features added
-------------------------
Removing digits
Removing pesky dots
Disabling emojis
-------------------------
Excess data removed
-------------------------
Tokenizing
Checking for stopwords
-------------------------
Data pre-processed
-------------------------


Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_135']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

...

Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count', 'questions', 'exclamations',
       'irony', 'vocab_richness', 'topic'],
      dtype='object')
Index(['index', 'source', 'author', 'title', 'description', 'url',
       'urlToImage', 'publishedAt', 'content', 'pre_processed_text',
       'minor_preprocessing', 'nrs_count',

In [120]:
model1.save('../big_picture/models/final_model')

In [94]:
model1.input_shape

In [112]:
model1.labels_tag = ohe.categories_[0]

In [113]:
model1.input_shape = 768

In [114]:
model1.output_shape = 16

In [115]:
model1.labels_tag

array(['Activism', 'Business', 'Crime', 'Culture', 'Education',
       'Entertainment', 'Health', 'Media', 'Other', 'Politics',
       'Religion', 'Science', 'Sports', 'Technology', 'Trends',
       'World News'], dtype=object)

In [121]:
new_model = Classifier()

In [122]:
new_model.load('../big_picture/models/final_model')

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_155']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'Activism': <big_picture.label.Label at 0x1b4043190>,
 'Business': <big_picture.label.Label at 0x1ae5e5d60>,
 'Crime': <big_picture.label.Label at 0x1afd4a3d0>,
 'Culture': <big_picture.label.Label at 0x186afbeb0>,
 'Education': <big_picture.label.Label at 0x184cadc70>,
 'Entertainment': <big_picture.label.Label at 0x185179280>,
 'Health': <big_picture.label.Label at 0x1867f3c10>,
 'Media': <big_picture.label.Label at 0x18320b6d0>,
 'Other': <big_picture.label.Label at 0x1abff60d0>,
 'Politics': <big_picture.label.Label at 0x1994acc70>,
 'Religion': <big_picture.label.Label at 0x18bd3f2e0>,
 'Science': <big_picture.label.Label at 0x1948a6f40>,
 'Sports': <big_picture.label.Label at 0x194c8ef40>,
 'Technology': <big_picture.label.Label at 0x18b21dcd0>,
 'Trends': <big_picture.label.Label at 0x191396070>,
 'World News': <big_picture.label.Label at 0x1970f1c10>}